# init

## install openpyxl

pandas uses this when saving Excel files. pandas imports it automatically, as needed.

In [1]:
!pip install openpyxl

## import Python modules

In [2]:
import pandas, IPython.core.display

## open file

In [3]:
src = pandas.ExcelFile('2017 CAM data from iPads.xlsx')
src.sheet_names

['2017 CAM data Erl',
 'schema (WIP reverse engineer)',
 '2017 CAM iPad data Tyler',
 'Combined iPad 2017 CAM data']

## select sheets

In [4]:
search_strings = ['Erl', 'Tyler']
sheets_to_process = {
    sheet_name.split(' ')[-1]: src.parse(sheet_name)
    for sheet_name in src.sheet_names
    if any(
        (
            (pattern in sheet_name) for pattern in search_strings
        )
    )
}
sheets_to_process.keys()

dict_keys(['Erl', 'Tyler'])

## Define Aphids and Natural Enemies

In [5]:
EGA = 'English grain'
BCO = 'bird cherry oat'
greenbug = 'green bug'
aphid_name_list = (EGA, BCO, greenbug)
aphid_name = {f'a{n + 1}': name for n, name in enumerate(aphid_name_list)}
aphid_name

{'a1': 'English grain', 'a2': 'bird cherry oat', 'a3': 'green bug'}

In [6]:
# @todo: input natural enemy names
natural_enemy_name_list = [
    'aphid_mummies_brown',
    'aphid_mummies_blk',
    'ladybeetle_larvae',
    'ladybeetle_adult',
    'lacewing_larvae',
    'lacewing_adult',
]
natural_enemy_name = {f'e{n + 1}': name for n, name in enumerate(natural_enemy_name_list)}
natural_enemy_name

{'e1': 'aphid_mummies_brown',
 'e2': 'aphid_mummies_blk',
 'e3': 'ladybeetle_larvae',
 'e4': 'ladybeetle_adult',
 'e5': 'lacewing_larvae',
 'e6': 'lacewing_adult'}

# cleanup

## normalize column names

In [216]:
sheets_needing_column_rename = (
    (sheet_name, sheet)
    for sheet_name, sheet
    in sheets_to_process.items()
    if any(
        (str(column).find(' ')
         for column in sheet.columns)
    )
)
sheets_to_process = {
    sheet_name: sheet.rename(
        mapper=lambda x: str(x).split(' ')[0],
        axis='columns'
    )
    for sheet_name, sheet in sheets_needing_column_rename
}

## concatenate sheets

In [217]:
concat_df = pandas.concat(
    sheets_to_process,
    names=['worksheet_name', 'index']
)

## reindex and rename columns

In [218]:
names = [
     'scope',       'group',         'variable',          'source_field']
columns = (
    ('field',       '',              'client',            'fields__client__displayText'),
    ('field',       'index',         'crop',              'fields__crop'),
    ('field',       'index',         'name',              'fields__name'),
    ('field',       'index',         'desc',              'fields__desc'),
    ('field',       '',              'date',              'fields__date'),
    ('set',         'index',         'date',              'fields__oSets__date'),
    ('set',         '',              'description',       'fields__oSets__desc'),
    ('set',         '',              'growth_stage',      'fields__oSets__growthStage'),
    ('set',         '',              'observer',          'fields__oSets__obsName'),
    ('set',         '',              'complete_sets',     'fields__oSets__completeSets'),
    ('set',         '',              'total_sets',        'fields__oSets__totalSets'),
    ('point',       'index',         'id',                'fields__oSets__oPoints__id'),
    ('observation', 'index',         'id',                'fields__oSets__oPoints__observations__id'),
    ('observation', '',              'complete',          'fields__oSets__oPoints__observations__complete'),
    ('observation', '',              'disabled',          'fields__oSets__oPoints__observations__disabled'),
    ('observation', '',              'natural_enemy_sub', 'fields__oSets__oPoints__observations__enum'),
    ('observation', '',              'aphid_subtotal',    'fields__oSets__oPoints__observations__anum'),
    ('observation', '',              'a1',                'fields__oSets__oPoints__observations__a1__number'),
    ('observation', '',              'a2',                'fields__oSets__oPoints__observations__a2__number'),
    ('observation', '',              'a3',                'fields__oSets__oPoints__observations__a3__number'),
    ('observation', 'natural_enemy', 'name',              'fields__oSets__oPoints__observations__|'),
    ('observation', 'natural_enemy', 'number',            'fields__oSets__oPoints__observations__|__number'),
)
columns = pandas.MultiIndex.from_tuples(columns, names=names)
df = concat_df.reindex(
    columns=columns,
    level='source_field')
df.head()

scope                                      field                            \
group                                                   index                
variable                                  client         crop         name   
source_field         fields__client__displayText fields__crop fields__name   
worksheet_name index                                                         
Erl            0                             NaN          NaN          NaN   
               1                             NaN          NaN          NaN   
               2                             NaN          NaN          NaN   
               3                             NaN          NaN          NaN   
               4                             NaN          NaN          NaN   

scope                                                               set  \
group                                                             index   
variable                     desc         date                     date   
source_field         fields__desc fields__date      fields__oSets__date   
worksheet_name index                                                      
Erl            0              NaN          NaN  2017-08-02T13:12:09.542   
               1              NaN          NaN                      NaN   
               2              NaN          NaN                      NaN   
               3              NaN          NaN                      NaN   
               4              NaN          NaN                      NaN   

scope                                                                \
group                                                                 
variable                     description               growth_stage   
source_field         fields__oSets__desc fields__oSets__growthStage   
worksheet_name index                                                  
Erl            0                     NaN                        7.0   
               1                     NaN                        NaN   
               2                     NaN                        NaN   
               3                     NaN                        NaN   
               4                     NaN                        NaN   

scope                                                                    \
group                                                                     
variable                           observer               complete_sets   
source_field         fields__oSets__obsName fields__oSets__completeSets   
worksheet_name index                                                      
Erl            0                      Tyler                         0.0   
               1                        NaN                         NaN   
               2                        NaN                         NaN   
               3                        NaN                         NaN   
               4                        NaN                         NaN   

scope                                      ...                        \
group                                      ...                         
variable                                   ...                         
source_field                               ...                         
worksheet_name index                       ...                         
Erl            0                           ...                         
               1                           ...                         
               2                           ...                         
               3                           ...                         
               4                           ...                         

scope                                             observation  \
group                                                   index   
variable                                                   id   
source_field         fields__oSets__oPoints__observations__id   
worksheet_name index     

## convert_datetime

In [219]:
date_column_mask, date_column_slice = df.columns.get_loc_level('date', level='variable')

In [220]:
df.loc[:, date_column_mask] = df.loc[:, date_column_mask].apply(pandas.to_datetime)
df.loc[:, date_column_mask].dropna(how='all').head()

scope                                  field                     set
group                                                          index
variable                                date                    date
source_field                    fields__date     fields__oSets__date
worksheet_name index                                                
Erl            0                         NaT 2017-08-02 13:12:09.542
               70    2017-08-09 09:24:11.845 2017-08-09 09:25:11.710
               140   2017-08-09 10:01:29.326 2017-08-09 10:06:25.480
               210   2017-08-09 11:16:15.922 2017-08-09 11:21:01.555
               350   2017-08-09 11:17:15.791 2017-08-09 11:37:20.862

## Define Common Index Columns

In [222]:
icols = df.columns.get_loc_level('index', level='group', drop_level=False)[1].values.tolist()
icols

[('field', 'index', 'crop', 'fields__crop'),
 ('field', 'index', 'name', 'fields__name'),
 ('field', 'index', 'desc', 'fields__desc'),
 ('set', 'index', 'date', 'fields__oSets__date'),
 ('point', 'index', 'id', 'fields__oSets__oPoints__id'),
 ('observation', 'index', 'id', 'fields__oSets__oPoints__observations__id')]

In [225]:
df[icols] = df[icols].ffill()

In [234]:
df = df.set_index(icols, append=True)
df.head(21)

scope                                                                                                                                                                                                                                                                                         field  \
group                                                                                                                                                                                                                                                                                                 
variable                                                                                                                                                                                                                                                                                     client   
source_field                                                                                                                                                                                                                                                            fields__client__displayText   
worksheet_name index (field, index, crop, fields__crop) (field, index, name, fields__name) (field, index, desc, fields__desc) (set, index, date, fields__oSets__date) (point, index, id, fields__oSets__oPoints__id) (observation, index, id, fields__oSets__oPoints...                               
Erl            0     NaN                                NaN                                NaN                                2017-08-02 13:12:09.542                 0.0                                            0.0                                                                        NaN   
               1     NaN                                NaN                                NaN                                2017-08-02 13:12:09.542                 0.0                                            1.0                                                                        NaN   
               2     NaN                                NaN                                NaN                                2017-08-02 13:12:09.542                 0.0                                            2.0                                                                        NaN   
               3     NaN                                NaN                                NaN                                2017-08-02 13:12:09.542                 0.0                                            3.0                                                                        NaN   
               4     NaN                                NaN                                NaN                                2017-08-02 13:12:09.542                 0.0                                            4.0                                                                        NaN   
               5     NaN                                NaN                                NaN                                2017-08-02 13:12:09.542                 0.0                                            5.0                                                                        NaN   
               6     NaN                                NaN                                NaN                                2017-08-02 13:12:09.542                 0.0                                            5.0                                                                        NaN   
               7     NaN                                NaN                                NaN                                2017-08-02 13:12:09.542                 0.0                                            5.0                                                                        NaN   
               8     NaN                                NaN                                NaN                                2017-08-02 13:12:09.542                

## unstack natural enemies

In [308]:
ne = (
    df
    .xs('natural_enemy', level='group', axis='columns', drop_level=False)
    .xs(5.0, level=7)
    .set_index(('observation', 'natural_enemy', 'name', 'fields__oSets__oPoints__observations__|'), append=True)
    .unstack()
    .rename(columns=natural_enemy_name, level=4)
    .loc[::9]
)
ne.head(15)

scope                                                                                                                                                                                                                                                       observation  \
group                                                                                                                                                                                                                                                     natural_enemy   
variable                                                                                                                                                                                                                                                         number   
source_field                                                                                                                                                                                                            fields__oSets__oPoints__observations__|__number   
('observation', 'natural_enemy', 'name', 'fields__oSets__oPoints__observations__|')                                                                                                                                                                 aphid_mummies_brown   
worksheet_name index (field, index, crop, fields__crop) (field, index, name, fields__name) (field, index, desc, fields__desc)    (set, index, date, fields__oSets__date) (point, index, id, fields__oSets__oPoints__id)                                                   
Erl            5     NaN                                NaN                                NaN                                   2017-08-02 13:12:09.542                 0.0                                                                                        NaN   
               19    NaN                                NaN                                NaN                                   2017-08-02 13:12:09.542                 1.0                                                                                        NaN   
               33    NaN                                NaN                                NaN                                   2017-08-02 13:12:09.542                 2.0                                                                                        NaN   
               47    NaN                                NaN                                NaN                                   2017-08-02 13:12:09.542                 3.0                                                                                        NaN   
               61    NaN                                NaN                                NaN                                   2017-08-02 13:12:09.542                 4.0                                                                                        NaN   
               75    CROPS.WHEAT                        Llewellyn wheat 1                  Midge susceptible wheat. Awned.       2017-08-09 09:25:11.710                 0.0                                                                                        NaN   
               89    CROPS.WHEAT                        Llewellyn wheat 1                  Midge susceptible wheat. Awned.       2017-08-09 09:25:11.710                 1.0                                                                                        NaN   
               103   CROPS.WHEAT                        Llewellyn wheat 1                  Midge susceptible wheat. Awned.       2017-08-09 09:25:11.710                 2.0                                                                                        NaN   
               117   CROPS.WHEAT                        Llewellyn wheat 1                  Midge susceptible wheat. Awned.       2017-08-09 09:25:11.710                 3.0                                                                                        Na

# explore

In [307]:
df.xs('', level='group', axis='columns', drop_level=False).head(15)

scope                                                                                                                                                                                                                                                                                         field  \
group                                                                                                                                                                                                                                                                                                 
variable                                                                                                                                                                                                                                                                                     client   
source_field                                                                                                                                                                                                                                                            fields__client__displayText   
worksheet_name index (field, index, crop, fields__crop) (field, index, name, fields__name) (field, index, desc, fields__desc) (set, index, date, fields__oSets__date) (point, index, id, fields__oSets__oPoints__id) (observation, index, id, fields__oSets__oPoints...                               
Erl            0     NaN                                NaN                                NaN                                2017-08-02 13:12:09.542                 0.0                                            0.0                                                                        NaN   
               1     NaN                                NaN                                NaN                                2017-08-02 13:12:09.542                 0.0                                            1.0                                                                        NaN   
               2     NaN                                NaN                                NaN                                2017-08-02 13:12:09.542                 0.0                                            2.0                                                                        NaN   
               3     NaN                                NaN                                NaN                                2017-08-02 13:12:09.542                 0.0                                            3.0                                                                        NaN   
               4     NaN                                NaN                                NaN                                2017-08-02 13:12:09.542                 0.0                                            4.0                                                                        NaN   
               5     NaN                                NaN                                NaN                                2017-08-02 13:12:09.542                 0.0                                            5.0                                                                        NaN   
               6     NaN                                NaN                                NaN                                2017-08-02 13:12:09.542                 0.0                                            5.0                                                                        NaN   
               7     NaN                                NaN                                NaN                                2017-08-02 13:12:09.542                 0.0                                            5.0                                                                        NaN   
               8     NaN                                NaN                                NaN                                2017-08-02 13:12:09.542                

# natural enemies

#### rename natural enemy columns

In [99]:
# ne2 = ne2.rename(columns=natural_enemy_name, level=1)
# ne2.head()

# save file

In [ ]:
# sum_by_point.columns = sum_by_point.columns.map('_'.join)
# joined_df.columns = joined_df.columns.map('_'.join)
# with pandas.ExcelWriter('out_df.xlsx') as file_writer:
#     sites_df.to_excel(
#         file_writer,
#         sheet_name='2017 - CAM - sites',
#         freeze_panes=(1, 1),
#         index_label='row'
#     )
#     sum_by_site.to_excel(
#         file_writer,
#         sheet_name='2017 - CAM - sum by site',
#         freeze_panes=(1, 1),
#         index_label='row'
#     )
#     sum_by_set.to_excel(
#         file_writer,
#         sheet_name='2017 - CAM - sum by time',
#         freeze_panes=(1, 1),
#         index_label='row'
#     )
#     sum_by_point.to_excel(
#         file_writer,
#         sheet_name='2017 - CAM - sum by point',
#         freeze_panes=(1, 1),
#         index_label='row'
#     )
#     joined_df.to_excel(
#         file_writer,
#         sheet_name='2017 - CAM - all observations',
#         freeze_panes=(1, 1),
#         index_label='row'
#     )
#     file_writer.save()